In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

/Users/asdf/miniforge3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:

df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")


df_train["TrainFlag"] = True
df_test["TrainFlag"] = False

df_all = df_train.append(df_test)
df_all.index = df_all["Id"]
df_all.drop("Id", axis = 1, inplace = True)

/var/folders/3c/7f_7lz2x20n0gd8mfd73s0m80000gn/T/ipykernel_5685/1177507823.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_all = df_train.append(df_test)


In [3]:
df_all = pd.get_dummies(df_all, drop_first=True)

In [4]:
df_train = df_all[df_all["TrainFlag"] == True]
df_train = df_train.drop(["TrainFlag"], axis = 1)

df_test = df_all[df_all["TrainFlag"] == False]
df_test = df_test.drop(["TrainFlag"], axis = 1)
df_test = df_test.drop(["SalePrice"], axis = 1)

In [5]:
y = df_train["SalePrice"].values
X = df_train.drop("SalePrice", axis=1).values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [6]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dvalid = xgb.DMatrix(X_test, label=y_test)
dtest = xgb.DMatrix(df_test.values)

In [7]:
params = {
        'objective': 'reg:squarederror','silent':1, 'random_state':1234, 
        
        'eval_metric': 'rmse',
    }
num_round = 500
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [8]:
model = xgb.train(params,
                    dtrain,
                    num_round,
                    early_stopping_rounds=20,
                    evals=watchlist,
                    )

[06:41:12] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117899018/work/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:140968.18750	eval-rmse:144201.06250
[1]	train-rmse:101726.89844	eval-rmse:106517.66406
[2]	train-rmse:73972.57812	eval-rmse:81385.10938
[3]	train-rmse:54587.18750	eval-rmse:63210.94531
[4]	train-rmse:40907.39453	eval-rmse:52497.31250
[5]	train-rmse:31129.76953	eval-rmse:45693.10547
[6]	train-rmse:24273.25391	eval-rmse:41899.23438
[7]	train-rmse:19573.60352	eval-rmse:39666.12500
[8]	train-rmse:16331.17773	eval-rmse:38170.98828
[9]	train-rmse:13992.89551	eval-rmse:37315.90234
[10]	train-rmse:12424.89453	eval-rmse:36737.40625
[11]	train-rmse:11342.17871	eval-r

In [9]:

prediction_XG = model.predict(dtest, ntree_limit = model.best_ntree_limit)

prediction_XG = np.round(prediction_XG)

/Users/asdf/miniforge3/lib/python3.9/site-packages/xgboost/core.py:105: UserWarning: ntree_limit is deprecated, use `iteration_range` or model slicing instead.
  warnings.warn(


In [10]:
prediction_XG

array([128539., 166515., 183189., ..., 170081., 116369., 236587.],
      dtype=float32)

In [11]:
submission = pd.DataFrame({"id": df_test.index, "SalePrice": prediction_XG})

In [12]:
submission

,id,SalePrice
0,1461,128539.0
1,1462,166515.0
2,1463,183189.0
3,1464,189899.0
4,1465,194400.0
...,...,...
1454,2915,84013.0
1455,2916,81581.0
1456,2917,170081.0
1457,2918,116369.0


In [22]:
y_test.shape

(438,)

In [18]:
pred

array([127281.01, 166943.72, 183379.94, ..., 171009.48, 116248.58,
       235646.16], dtype=float32)

In [21]:
pred.shape

(1459,)